In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from glob import glob
import matplotlib.pyplot as plt
import json

In [4]:
logfiles = sorted(glob('outputs/*'))
print(len(logfiles))

110


In [5]:
pltfiles = sorted(glob('images/plots/*'))
print(len(pltfiles))

109


In [6]:
nTargets = 6
nFolds = 9

In [39]:
modes = ['ours', 'ours_w']
meta = {}
for group in modes: 
    meta[group] = [None] * nTargets
    for tii in range(nTargets):
        meta[group][tii] = [None] *nFolds

mcount = {}
for md in modes: mcount[md] = 0
for lf in logfiles:
    mode = lf.split('-')[0].split('/')[1]
    if mode not in modes: continue
        
    mcount[mode] += 1
        
    target = int(lf.split('targ_')[1][0])
    fold = int(lf.split('fold_')[1][0])

    with open(lf) as fl:
        hist = [json.loads(line) for line in fl.read().split('\n') if line]
    ent = dict(log=lf, plot=pf, history=hist)
#     print(mode, target, fold)
    if meta[mode][target][fold] is not None:
        
        raise Exception('Indexing error')
    meta[mode][target][fold] = ent
    
print(mcount)

{'ours': 54, 'ours_w': 54}


In [34]:
def fold_avg_prop(mode, fold, prop):
    agg = 0
    for target in meta[mode]:
#         print(targ)
        val = min([ent[prop] for ent in target[fold]['history']])
        agg += val
#     print(agg)
    return agg / nTargets

In [51]:
PROPS = ['MSE_TEST', 'MAPE_TEST']
import numpy as np

def table(mode, printout=False):
    mdstr = ' (%s)' % mode
    mse = [fold_avg_prop(mode, ii, 'MSE_TEST') for ii in range(nFolds)]
    mape = [fold_avg_prop(mode, ii, 'MAPE') for ii in range(nFolds)]
    
    mseline = '\t'.join(['MSE%s' % mdstr] \
                    + ['%.2f' % val for val in mse] + ['%.2f' % np.mean(mse)])
    mapeline = '\t'.join(['MAPE%s' % mdstr] \
                    + ['%.2f' % val for val in mape] + ['%.2f' % np.mean(mape)])
    
    return [mseline, mapeline]

In [55]:
header = '\t'.join(['metric'] + ['fold-%d' % ii for ii in range(1, 10)] + ['fold-avg'])
lines = table('ours') + table('ours_w')

print(header)
print(lines[0])
print(lines[2])
print(lines[1])
print(lines[3])

with open('table-fold.tsv', 'w') as fl:
    fl.write(header + '\n')
    fl.write(lines[0] + '\n')
    fl.write(lines[2] + '\n')
    fl.write(lines[1] + '\n')
    fl.write(lines[3] + '\n')

metric	fold-1	fold-2	fold-3	fold-4	fold-5	fold-6	fold-7	fold-8	fold-9	fold-avg
MSE (ours)	631.77	167.52	332.37	276.03	230.27	527.06	89.54	284.00	182.43	302.33
MSE (ours_w)	622.80	168.15	332.52	268.47	230.73	529.85	88.87	286.82	202.25	303.38
MAPE (ours)	16.89	16.82	18.55	15.76	19.18	16.94	12.43	9.12	15.08	15.64
MAPE (ours_w)	16.99	16.65	18.59	15.43	19.39	16.88	12.16	9.17	15.87	15.68
